Did you remember to run the bash script before opening up the Jupyter Notebook?

`. ~/Desktop/sql_variables.sh`

In [1]:
import pymysql
import pandas as pd
import os

In [2]:
conn = pymysql.connect(host=os.environ.get('SQLHOSTID'),
                       port=3306,
                       user=os.environ.get('SQLUSERID'),
                       passwd=os.environ.get('SQLPASSWORD'),
                       db='financial')

In [3]:
loan_query = '''
select account.account_id, disp_O.client_id as owner_client_id, disp_O.disp_id as owner_disp_id, 
        disp_num.num_acct_users, client.birth_date as owner_birth_date, account.frequency as statement_freq, 
        card.type as card_type, card.issued as card_issue_date, account.date as acct_start_date, loan.loan_id, 
        loan.date as loan_date, loan.amount as loan_amt, payments.total_paid, loan.duration as loan_duration_mos, 
        payments.total_payments, loan.payments as monthly_payments, loan.status as loan_status, 
        account.district_id, district.A2, district.A3, district.A4, district.A5, district.A6, district.A7, 
        district.A8, district.A9, district.A10, district.A11, district.A12, district.A13, district.A14, 
        district.A15, district.A16
from loan
left join account on loan.account_id = account.account_id
left join district on account.district_id = district.district_id
left join (select account_id, sum(amount) as total_paid, count(date) as total_payments
            from financial.trans
            where k_symbol like 'UVER'
            group by account_id) as payments on account.account_id = payments.account_id
left join (select * from disp
            where disp.type like 'O%') as disp_O on loan.account_id = disp_O.account_id
left join client on disp_O.client_id = client.client_id
left join card on disp_O.disp_id = card.disp_id
left join (select account_id, count(client_id) as num_acct_users
            from disp
            group by account_id) as disp_num on loan.account_id = disp_num.account_id;'''

In [4]:
account_query = '''
select account.account_id, disp_O.client_id as owner_client_id, disp_O.disp_id as owner_disp_id, 
        disp_num.num_acct_users, client.birth_date as owner_birth_date, account.frequency as statement_freq, 
        card.type as card_type, card.issued as card_issue_date, account.date as acct_start_date, 
        account.district_id, district.A2, district.A3, district.A4, district.A5, district.A6, district.A7, 
        district.A8, district.A9, district.A10, district.A11, district.A12, district.A13, district.A14, 
        district.A15, district.A16
from account
left join district on account.district_id = district.district_id
left join (select * from disp
            where disp.type like 'O%') as disp_O on account.account_id = disp_O.account_id
left join client on disp_O.client_id = client.client_id
left join card on disp_O.disp_id = card.disp_id
left join (select account_id, count(client_id) as num_acct_users
            from disp
            group by account_id) as disp_num on account.account_id = disp_num.account_id;'''

In [5]:
payments_query = '''
select account_id, k_symbol, count(trans_id), sum(amount) 
from financial.trans
where k_symbol like 'POJISTNE' or k_symbol like 'UVER' or k_symbol like 'SIPO' or k_symbol like 'LEASING'
group by account_id, k_symbol;
'''

In [6]:
loan_df = pd.read_sql(loan_query, conn)

In [8]:
account_df = pd.read_sql(account_query, conn)

In [9]:
payment_df = pd.read_sql(payments_query, conn)

In [10]:
# close connection after you get what you want
conn.close()

In [11]:
loan_df.to_csv('loan_info.csv', index=False)
account_df.to_csv('account_info.csv', index=False)
payment_df.to_csv('payment_info.csv', index=False)